## Agile Hardware Design
***
# Encapsulation

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Winter22/)

## Plan for Today

* Scala methods, recursion, and objects
* Using Scala methods to build Chisel components
* Chisel Bundles

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Scala Methods

* Syntax examples on right
  * Multi-line bodies need braces
  * Last line is returned
  * Normally inside `class` or `object`

* Arguments are immutable by default

* Can give default argument values


In [ ]:
def plusOne(n: Int) =  n + 1

plusOne(5)

def plusX(n: Int, x: Int = 1) = n + x

plusX(5,2)
plusX(5)

## Recursive Scala Methods


* Need be sure to specify return type
  * `Unit` is nothing (like void)
* Be sure to think of base case
* Helpful for iteration or decomposing a problem


In [ ]:
def recSum(n: Int): Int = {
    if (n <= 0) 0
    else n + recSum(n - 1)
}

println(recSum(4))

// def fib(n: Int): Int = {
//     if (n < 2) n
//     else fib(n-1) + fib(n-2)
// }

// for (n <- 0 until 10)
//     println(fib(n))

## Using Scala to Construct Chisel Components

* Chisel components are just objects in Scala, so can use methods to build them up
  * Can _encapsulate_ (hide) complexity
  * Can declare once, use in many places
  * Use recursion to perform iteration (to implement paramterized flexibility)

* Although testers requires a `Module`, valid to construct components outside
  * Chisel's `Module` & `Bundle` add things to class (through inheritance)

## DelayN (shift register) Revised with Helper Function

In [ ]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n >= 0)
    var lastConn = io.in
    for (i <- 0 until n)
        lastConn = RegNext(lastConn)
    io.out := lastConn
}
println(getVerilog(new DelayNCycles(2)))

In [ ]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n >= 0)
    def helper(n: Int, lastConn: Bool): Bool = {
        if (n == 0) lastConn
        else helper(n-1, RegNext(lastConn))
    }
    io.out := helper(n, io.in)
}
println(getVerilog(new DelayNCycles(2)))

## Scala `object`

* _Singleton_ object
  * Exacly one always exists
  * By contrast, must instantiate a `class` to use

* Typical uses
  * Shared state (constant or mutable)
  * Stateless functions
  * Factory methods (as _companion object_)

In [ ]:
class MyPair(a: Int, b: Int) {
    def sum() = a + b
}

val mpc = new MyPair(3,4)
mpc.sum()
mpc.sum

object MyPair {
    var numPairs = 0
    def apply(a: Int, b: Int) = {
        numPairs += 1
        new MyPair(a,b)
    }
    def apply(a: Int): MyPair = apply(a, 0)
}

MyPair(2,3).sum
MyPair.numPairs
val mpo = MyPair(3)
mpo.sum
MyPair.numPairs

## Factory Method for `MyCounter`

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
    io.out := count
}

object MyCounter {
    def apply(maxVal: Int) = new MyCounter(maxVal)
}

In [ ]:
println(getVerilog(new MyCounter(15)))
// println(getVerilog(MyCounter(15)))

## MyCounter without `Module`

In [ ]:
class MyCounter(maxVal: Int, en: Bool) {
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
}

object MyCounter {
    def apply(maxVal: Int, en: Bool) = {
        val mc = new MyCounter(maxVal, en)
        mc.count
    }
}

In [ ]:
class CounterInstMod(n: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val count = Output(UInt())
    })
    io.count := MyCounter(n, io.en)
}
println(getVerilog(new CounterInstMod(4)))

## Chisel's Counter

* Prior slides only an example, Chisel defines [`Counter`](https://www.chisel-lang.org/api/latest/chisel3/util/Counter.html)

In [ ]:
class CounterInstMod(n: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val count = Output(UInt())
        val limit = Output(Bool())
    })
    val (value, wrap) = Counter(io.en, n)
//     val (value, wrap) = Counter(0 until 7 by 2, io.en)
    io.count := value
    io.limit := wrap
}

In [ ]:
println(getVerilog(new CounterInstMod(4)))

## Chisel `Bundle`

* Aggregate type with named fields
  * Like a `struct` in C
* Not only used to make IO interfaces
* Can declare once & use in many places

In [ ]:
class Mag extends Bundle {
    val m = UInt(4.W)
}

class OutMod(a: Int) extends Module {
    val io = IO(Output(new Mag))
    io.m := a.U
}

println(getVerilog(new OutMod(2)))

## `Bundle` Composition

* Bundles can ...
  * be _extended_
  * be nested
  * go in `Vec`s or include `Vec`s


In [ ]:
class Mag extends Bundle {
    val m = Output(UInt(4.W))
}

class SignMag extends Mag {
    val s = Output(Bool())
}

class PairSignMag extends Bundle {
    val nums = Vec(2, new SignMag)
}

class OutMod(a: Int, b: Int) extends Module {
    val io = IO(new PairSignMag)
    io.nums(0).m := a.U
    io.nums(0).s := false.B
    io.nums(1).m := b.U
    io.nums(1).s := false.B
}

println(getVerilog(new OutMod(3,4)))

## `cloneType` Boilerplate

* Unfortunately, due to embedding Chisel in Scala, had to include `cloneType`
  * Most often occurs when parameterizing a Bundle
* ~~Future versions of Chisel hope to have smoother work around~~
    * **Chisel 3.5 fixed this!**

In [ ]:
class SignMag(n: Int) extends Bundle {
    val x = Output(UInt(n.W))
    val s = Output(Bool())
    // No longer necessary
    // override def cloneType = (new SignMag(n)).asInstanceOf[this.type]
}

class OutMod(n: Int, a: Int) extends Module {
    val io = IO(Output(new SignMag(8)))
    io.x := a.U
    io.s := false.B
}

println(getVerilog(new OutMod(8,4)))

## Working With Bundles Hierarchically

* `<>` is a _bulk connection_
    * Will connect an entire Bundle

In [ ]:
class SignMag(n: Int) extends Bundle {
    val x = UInt(n.W)
    val s = Bool()
}

class PassThru(n: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(new SignMag(n))
        val out = Output(new SignMag(n))
    })
//     io.out.x := io.in.x
//     io.out.s := io.in.s
    io.in <> io.out
}

println(getVerilog(new PassThru(4)))

## Bundles Can Have Wires in Both Directions

* `Flipped` reverses directions



In [ ]:
class Handshake(n: Int) extends Bundle {
    val ready = Input(Bool())
    val data  = Output(UInt(n.W))
}

class PassThru(n: Int) extends Module {
    val io = IO(new Bundle {
        val in = Flipped(new Handshake(n))
        val out = new Handshake(n)
    })
    io.in <> io.out
}

println(getVerilog(new PassThru(4)))

## Scala `Option`

* Scala's `Option[T]` is a type wrapper around `T` to indicate potential non-existence of something
  * Is either `None` or `Some(x)`
* Brief Option API primer (will learn more graceful methods later)
  * `isDefined` - returns `Boolean` indicating if it is has something
  * `get` - returns value if it has something, otherwise exception
* Can use Option to have optional fields in a Bundle

In [ ]:
val o: Option[Int] = Some(4)
// val o: Option[Int] = None
if (o.isDefined)
    println(o.get)
else
    println("empty")

## Making Optional IOs in Chisel

In [ ]:
class MaybePair(n: Int, hasY: Boolean) extends Bundle {
    val x = Output(UInt(n.W))
    val y: Option[UInt] = if (hasY) Some(Output(UInt(n.W))) else None
}

class OutMod(n: Int, a: Int, useY: Boolean) extends Module {
    val io = IO(Output(new MaybePair(8, useY)))
    io.x := a.U
    if (useY)
//     if (io.y.isDefined)
        io.y.get := a.U
}

println(getVerilog(new OutMod(8,4,true)))

## Scala `tabulate`

* May have seen this method used in assignments
* More general way (than `fill`) to populate a collection
* Will produce new collection by calling _anonymous function_ on every element
  * This function takes a single argument (index)
  * Can use `_` to wildcard replace first use, but be careful

In [ ]:
Seq.fill(4)(0)
Seq.tabulate(4)(i => i)
Seq.tabulate(4)(_*2)